In [1]:
!pip install datasets

!pip install transformers

!pip install accelerate -U
!pip install transformers[torch]
!pip install torch -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [1]:
from datasets import Dataset, DatasetDict
import pandas as pd
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoTokenizer

In [ ]:
notebook_login()

In [ ]:
notebook_login()

In [2]:
dataset = load_dataset('davidgaofc/MedQuad_split')

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating SFT_train1 split:   0%|          | 0/5742 [00:00<?, ? examples/s]

Generating SFT_train2 split:   0%|          | 0/5742 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1640 [00:00<?, ? examples/s]

Generating RL split:   0%|          | 0/1640 [00:00<?, ? examples/s]

Generating RM_oos split:   0%|          | 0/820 [00:00<?, ? examples/s]

Generating Shadow_oos split:   0%|          | 0/820 [00:00<?, ? examples/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [4]:
def preprocess_function(examples):
    input_encodings = tokenizer(examples["Question"], truncation=True, padding="max_length")
    target_encodings = tokenizer(examples['Answer'], truncation=True, padding="max_length")
    input_encodings["labels"] = target_encodings["input_ids"]
    return input_encodings

tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/5742 [00:00<?, ? examples/s]

Map:   0%|          | 0/5742 [00:00<?, ? examples/s]

Map:   0%|          | 0/1640 [00:00<?, ? examples/s]

Map:   0%|          | 0/1640 [00:00<?, ? examples/s]

Map:   0%|          | 0/820 [00:00<?, ? examples/s]

Map:   0%|          | 0/820 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoConfig
config = AutoConfig.from_pretrained("t5-small")
config.max_length = 200
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small", config=config)

In [12]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small")

In [7]:
!pip install evaluate rouge_score transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=db92da8344d22e6c3909c24c325e43308f2a60479e7dbd047118ed96fc406add
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 3.7 MB/s eta 0:00:00


In [13]:
import evaluate
import numpy as np

metric = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# metric = evaluate.load("sacrebleu")

# def postprocess_text(preds, labels):
#     preds = [pred.strip() for pred in preds]
#     labels = [[label.strip()] for label in labels]

#     return preds, labels

# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

#     result = metric.compute(predictions=decoded_preds, references=decoded_labels)
#     result = {"bleu": result["score"]}

#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
#     result["gen_len"] = np.mean(prediction_lens)
#     result = {k: round(v, 4) for k, v in result.items()}
#     return result

In [14]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results2",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["SFT_train1"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [15]:
import torch

torch.cuda.empty_cache()

In [16]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.651500,1.519522,0.009900,0.002500,0.008700,0.008600,11.318900
2,1.731000,1.392032,0.168900,0.050000,0.136700,0.136600,138.998800
3,1.541500,1.323172,0.206000,0.060100,0.164200,0.164200,153.786000
4,1.499300,1.286469,0.208200,0.062200,0.165100,0.165100,151.953000
5,1.450200,1.264027,0.236600,0.087000,0.188300,0.188400,153.762800
6,1.422600,1.249138,0.252600,0.108300,0.205300,0.205700,154.090200
7,1.417500,1.238452,0.265400,0.118300,0.216800,0.217100,152.617100
8,1.385500,1.231920,0.266100,0.118000,0.218400,0.218500,153.408500
9,1.395600,1.227882,0.268500,0.119400,0.220700,0.220800,154.528000
10,1.397800,1.226575,0.268200,0.119400,0.220800,0.221000,154.522600


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


TrainOutput(global_step=7180, training_loss=1.5500358050248084, metrics={'train_runtime': 9205.6432, 'train_samples_per_second': 6.237, 'train_steps_per_second': 0.78, 'total_flos': 7771326240522240.0, 'train_loss': 1.5500358050248084, 'epoch': 10.0})

In [17]:
notebook_login()

In [18]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

events.out.tfevents.1700674812.dcf2c99129d6.1851.0:   0%|          | 0.00/13.0k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

'https://huggingface.co/davidgaofc/results2/tree/main/'

In [ ]:
# from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer


In [ ]:
# model = AutoModelForSeq2SeqLM.from_pretrained("davidgaofc/SFT_Med")

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# training_args = Seq2SeqTrainingArguments(
#     output_dir="./results2",
#     evaluation_strategy="epoch",
#     per_device_eval_batch_size=1,
#     fp16=True,
# )

# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     # train_dataset=tokenized_dataset["SFT_train1"],
#     eval_dataset=tokenized_dataset["test"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )

In [ ]:
trainer.evaluate()

OutOfMemoryError: ignored